In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import numpy as np

from CleaningFunctions import *

warnings.filterwarnings("ignore")

In [8]:
#Planilhas a serem lidas

month_FormerGC = ['10_Medição_GC_Outubro_2017_(SGM)','11_Medição_GC_Novembro_2017_(SGM)','12_Medição_GC_Dezembro_2017_(SGM)',
                 '01_Medição_GC_Janeiro_2018_(SGM)','02_Medição_GC_Fevereiro_2018_(SGM)','03_Medição_GC_Março_2018_(SGM)','04_Medição_GC_Abril_2018_(SGM)','05_Medição_GC_Maio_2018_(SGM)','06_Medição_GC_Junho_2018_(SGM)','07_Medição_GC_Julho_2018_(SGM)','08_Medição_GC_Agosto_2018_(SGM)','09_Medição_GC_Setembro_2018_(SGM)','10_Medição_GC_Outubro_2018_(SGM)','11_Medição_GC_Novembro_2018_(SGM)','12_Medição_GC_Dezembro_2018_(SGM)',
                 '01_Medição_GC_Janeiro_2019_(SGM)','02_Medição_GC_Fevereiro_2019_(SGM)','03_Medição_GC_Março_2019_(SGM)','04_Medição_GC_Abril_2019_(SGM)','05_Medição_GC_Maio_2019_(SGM)','06_Medição_GC_Junho_2019_(SGM)','07_Medição_GC_Julho_2019_(SGM)','08_Medição_GC_Agosto_2019_(SGM)','09_Medição_GC_Setembro_2019_(SGM)','10_Medição_GC_Outubro_2019_(SGM)','11_Medição_GC_Novembro_2019_(SGM)','12_Medição_GC_Dezembro_2019_(SGM)',
                 '01_Medição_GC_Janeiro_2020_(SGM)','02_Medição_GC_Fevereiro_2020_(SGM)','03_Medição_GC_Março_2020_(SGM)','04_Medição_GC_Abril_2020_(SGM)','05_Medição_GC_Maio_2020_(SGM)','06_Medição_GC_Junho_2020_(SGM)','07_Medição_GC_Julho_2020_(SGM)','08_Medição_GC_Agosto_2020_(SGM)','09_Medição_GC_Setembro_2020_(SGM)','10_Medição_GC_Outubro_2020_(SGM)','11_Medição_GC_Novembro_2020_(SGM)']

month_NewGC = ['12_Medição_GC_Dezembro_2020_(SGM)','01-21 - GC Planilha Fiscal - Janeiro','02-21 - GC Planilha Fiscal - Fevereiro','03-21 - GC Planilha Fiscal - Março','04-21 - GC Planilha Fiscal - Abril','05-21 - GC Planilha Fiscal - Maio','06-21 - GC Planilha Fiscal - Junho','07-21 - GC - Planilha Fiscal - Julho','08-21 - GC - Planilha Fiscal - Agosto']

In [5]:
def leituraRelMed3(listaDeDF, formato, abaPlanilha, cabecalho):
    dataframes_list = []

    for i in range(len(listaDeDF)):
        temp_df = pd.read_excel(listaDeDF[i]+formato,sheet_name = abaPlanilha,header = cabecalho)
        
        #limpeza do cabeçalho
        temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed', na = True)]
        temp_df.columns = temp_df.columns.str.strip()
        temp_df.columns = temp_df.columns.str.replace(r'\n', r' ',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'@', r'',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'\s+', r' ',regex = True)

        #Strip todos os elementos do df
        temp_df = temp_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        
        #Remoção das linhas invalidas
        temp_df = temp_df[temp_df['Dia'].notna()]
        temp_df = temp_df[temp_df['Dia'].apply(lambda x: isinstance(x, str))]
        temp_df = temp_df.drop_duplicates(subset=['Dia', 'Dia'], keep='first')

        #Seleciono Colunas de interesse
        temp_df = temp_df[['Dia','Volume Total Observado (m³)','Temperatura do Fluido em Tanque','Temperatura do Ambiente','Temperatura da Amostra em Laboratório','Fator de Expansão da Chapa','Grau API Lido','Grau API ( 60 °F)','Densidade (g/cm³ 20°C e 1 atm)','Fator de Expansão do Líquido','BS&W (%)','Água e Sedimentos (m³)','Volume Líquido (m³ 20°C & 1 atm)']]
        
        #Converção da temperatura do laboratorio de Fº para Cº
        temp_df['Temperatura da Amostra em Laboratório'] = (temp_df['Temperatura da Amostra em Laboratório'] - 32) / 1.8

        #Remoção das datas invalidas
        temp_df = temp_df[(temp_df['Dia'] != '31/11/17')&(temp_df['Dia'] != '31/04/18')&(temp_df['Dia'] != '29/02/19')&(temp_df['Dia'] != '30/02/19')&(temp_df['Dia'] != '31/02/19')&(temp_df['Dia'] != '31/04/19')&(temp_df['Dia'] != '31/06/19')&(temp_df['Dia'] != '31/09/19')&(temp_df['Dia'] != '31/11/19')&(temp_df['Dia'] != '30/02/20')&(temp_df['Dia'] != '30/02/20')&(temp_df['Dia'] != '31/02/20')&(temp_df['Dia'] != '31/04/20')&(temp_df['Dia'] != '31/06/20')&(temp_df['Dia'] != '31/09/20')&(temp_df['Dia'] != '31/11/20')]

        #transformo data para viriavel tipo "datetime"
        temp_df['Dia'] = pd.to_datetime(temp_df['Dia'], format="%d/%m/%y").dt.strftime("%Y-%m-%d")
        
        dataframes_list.append(temp_df)
        
    foo = pd.concat(dataframes_list)
    foo = foo.reset_index(drop = True)
    
    return foo


def leituraRelMed2(listaDeDF, formato, abaPlanilha, cabecalho):

    for i in range(len(listaDeDF)):
        temp_df = pd.read_excel(listaDeDF[i]+formato,sheet_name = abaPlanilha,header = cabecalho)
        
        #limpeza do cabeçalho
        temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed', na = True)]
        temp_df.columns = temp_df.columns.str.strip()
        temp_df.columns = temp_df.columns.str.replace(r'\n', r' ',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'@', r'',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'\s+', r' ',regex = True)

        temp_df = temp_df.rename(columns={"Data":"Dia"})

        #Strip todos os elementos do df
        temp_df = temp_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        
        #Remoção das linhas inválidas
        temp_df = temp_df[temp_df['Dia'] != 'TOTAIS']
        temp_df = temp_df[temp_df['Dia'].notna()]
        temp_df = temp_df[temp_df['Dia'].apply(lambda x: isinstance(x, str))]
        temp_df = temp_df.drop_duplicates(subset=['Dia', 'Dia'], keep='first')

        #Removo datas inexistente
        temp_df = temp_df[(temp_df['Dia'] != '31/11/17')&(temp_df['Dia'] != '31/04/18')&(temp_df['Dia'] != '29/02/19')&(temp_df['Dia'] != '30/02/19')&(temp_df['Dia'] != '31/02/19')&(temp_df['Dia'] != '31/04/19')&(temp_df['Dia'] != '31/06/19')&(temp_df['Dia'] != '31/09/19')&(temp_df['Dia'] != '31/11/19')&(temp_df['Dia'] != '30/02/20')&(temp_df['Dia'] != '30/02/20')&(temp_df['Dia'] != '31/02/20')&(temp_df['Dia'] != '31/04/20')&(temp_df['Dia'] != '31/06/20')&(temp_df['Dia'] != '31/09/20')&(temp_df['Dia'] != '31/11/20')]

        temp_df = temp_df.reset_index(drop = True)

        #transformo data para viriavel tipo "datetime"
        temp_df['Dia'] = pd.to_datetime(temp_df['Dia'], format="%d/%m/%y").dt.strftime("%Y-%m-%d")
        
        dataframes_list.append(temp_df)
        
    foo = pd.concat(dataframes_list)
    foo = foo.reset_index(drop = True)
    
    return foo

        
#leitura Gas
def leituraGas(listaDeDF, formato, abaPlanilha, cabecalho):

    for i in range(len(listaDeDF)):
        temp_df = pd.read_excel(listaDeDF[i]+formato,sheet_name = abaPlanilha,header = cabecalho)
        
        year = str(temp_df.iat[0, 1])
        month = temp_df.iat[0, 3]
        month = month.lower()
        month = month_name(month)
        temp_df.columns = temp_df.iloc[6]
        temp_df.columns.values[0] = "Data"
        temp_df["Data"] = temp_df["Data"].astype(str) + "/" + month + "/" + year
        temp_df = temp_df.iloc[8:39, :]
        temp_df = temp_df[temp_df.columns.dropna()]
        
        temp_df.columns = temp_df.columns.str.replace(r'\n', r' ',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'\s+', r' ',regex = True)
        
        #Selecionar colunas de interesse
        temp_df = temp_df[['Data','Gás Total (20°C e 1atm) (m³):','Água Produzida (m³):', 'Gás Fiscal de Queima (20°C e 1atm) (m³):','Gás Fiscal de Compressão (20°C e 1atm) (m³):']]

        #Remoção e renomeação das linhas invalidas
        temp_df = temp_df.drop_duplicates(subset=['Data', 'Data'], keep='first')

        #Removo datas inexistente
        temp_df = dataInvalida(temp_df)

        #transformo data para viriavel tipo "datetime"
        temp_df['Data'] = pd.to_datetime(temp_df['Data'], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")

        temp_df = temp_df.rename(columns={"Data":"Dia"})
        
        dataframes_list.append(temp_df)
    
    foo = pd.concat(dataframes_list)
    
    foo = foo.reset_index(drop = True)
    return foo

        
def newFechamentoFiscal(listaDeDF, formato, abaPlanilha, cabecalho):
    
    for i in range(len(listaDeDF)):
        temp_df = pd.read_excel(listaDeDF[i]+formato,sheet_name = abaPlanilha,header = cabecalho)
        #Remover colunas em branco e sem nome
        temp_df = temp_df[temp_df['DATA'].notna()]
        temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed')]

        #Redefinir o index
        temp_df = temp_df.reset_index(drop = True)

        #Elimino duplicatas
        temp_df = temp_df.drop_duplicates(subset=['DATA', 'DATA'], keep='first')

        #Removo porcentagem
        temp_df['BSW %'] = temp_df['BSW %']*100

        temp_df.columns = temp_df.columns.str.replace(r'\n', r' ',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'\s+', r' ',regex = True)

        #Seleciono colunas de interesse
        temp_df = temp_df[['DATA','Volume bruto observado (GOV-m³)','Temperatura do líquido (°C)','Temperatura ambiente (°C)','Temperatura de Laboratório (°C)','Correção do efeito da temperatura no aço (CTSh)','API lido','API ( 60° F)','Densidade Corrigida','FCV','BSW %','Água na emulsão (m³)','NSV - Volume líquido nas condições padrões 20° C e 1 atm','Prod. Líquida dia (m³)']]

        temp_df['DATA'] = pd.to_datetime(temp_df['DATA']).dt.date

        dataframes_list.append(temp_df)
        
    foo = pd.concat(dataframes_list)
    foo = foo.iloc[:-1 , :]

    foo = foo.reset_index(drop = True)
    return foo
        
def leituraGasNovo(listaDeDF, formato, abaPlanilha, cabecalho):
    
    for i in range(len(listaDeDF)):
        temp_df = pd.read_excel(listaDeDF[i]+formato,sheet_name = abaPlanilha,header = cabecalho)
        
        temp_df.columns = temp_df.columns.str.replace(r'\n', r' ',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'\s+', r' ',regex = True)

        temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed')]

        temp_df = temp_df[['DATA','Gás Fiscal (20°C e 1atm) (m³):','Gás Fiscal de Queima (20°C e 1atm) (m³):','Gás Fiscal de Compressão (20°C e 1atm) (m³):','Medição (20°C e 1 atm) (m³)']]
        temp_df = temp_df.dropna(subset=['DATA'])
        
        temp_df['DATA'] = temp_df['DATA'].astype('str')
        temp_df['DATA'] = pd.to_datetime(temp_df['DATA']).dt.date
        
        dataframes_list.append(temp_df)
    
    foo = pd.concat(dataframes_list)

    foo = foo.reset_index(drop = True)
    return foo

<h2> Planilhas antigas <h2>

In [4]:
%cd "C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Galo Campina\GC Antigo"

C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Galo Campina\GC Antigo


<h3>Aba - RelMedFisc2

In [5]:
dataframes_list = []
formerVolLiquidoGC = leituraRelMed2(month_FormerGC,'.xlsm', 'RelMedFisc2',4)

#Seleção das colunas de interesse!
formerVolLiquidoGC = formerVolLiquidoGC[['Dia','Volume Líquido (m³ 20°C & 1 atm).2']]

In [6]:
#formerVolLiquidoGC = formerVolLiquidoGC.rename(columns={"Volume Líquido (m³ 20°C & 1 atm).2":"Prod. Líquida dia (m³)"})

<h3>Aba - RelMedFisc3

In [8]:
dataframes_list = []
formerFechamentoFiscalGC = leituraRelMed3(month_FormerGC,'.xlsm','RelMedFisc3',5)

<h3>Aba - Fechamento da Produção

In [9]:
dataframes_list = []
formerGas_GC = leituraGas(month_FormerGC,'.xlsm', 'Fechamento de Produção',4)

<h3> Agregar das planilhas antigas

In [56]:
#Agregar planilhas de outubro/2017 até novembro/2020
fechamento_fiscal_GC = pd.merge(formerVolLiquidoGC, formerFechamentoFiscalGC, how="outer")
fechamento_fiscal_GC = pd.merge(fechamento_fiscal_GC, formerGas_GC, how="outer")

#Padronizar colunas
fechamento_fiscal_GC = fechamento_fiscal_GC.rename(columns={"Dia":"DATA","Volume Líquido (m³ 20°C & 1 atm).2":"Prod. Líquida dia (m³)","Volume Total Observado (m³)":"Volume bruto observado (GOV-m³)","Temperatura do Fluido em Tanque":"Temperatura do líquido (°C)","Temperatura do Ambiente":"Temperatura ambiente (°C)","Temperatura da Amostra em Laboratório":"Temperatura de Laboratório (°C)","Fator de Expansão da Chapa":"Correção do efeito da temperatura no aço (CTSh)","Grau API Lido":"API lido","Grau API ( 60 °F)":"API ( 60° F)","Densidade (g/cm³ 20°C e 1 atm)":"Densidade Corrigida","Fator de Expansão do Líquido":"FCV","BS&W (%)":"BSW %","Água e Sedimentos (m³)":"Água na emulsão (m³)","Volume Líquido (m³ 20°C & 1 atm)":"NSV - Volume líquido nas condições padrões 20° C e 1 atm","Volume_líquido_Produção":"Prod. Líquida dia (m³)","Gás Total (20°C e 1atm) (m³):":"Gás Fiscal (20°C e 1atm) (m³):","Água Produzida (m³):":"Medição (20°C e 1 atm) (m³)"})

<h1> Planilhas novas <h1>

In [9]:
%cd "C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Galo Campina\GC Novo"

C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Galo Campina\GC Novo


<h3>Aba - Fechamento fiscal

In [10]:
dataframes_list = []
novoFechamentoFiscalGC = newFechamentoFiscal(month_NewGC,'.xlsx','Fechamento fiscal',3)

<h3>Aba - Gás Fiscal

In [11]:
dataframes_list = []
novoGasGC2021 = leituraGasNovo(month_NewGC,".xlsx","Gás fiscal",2)

<h3> Agregar novas planilhas

In [13]:
fF_novo = pd.merge(novoFechamentoFiscalGC, novoGasGC2021, how="outer")

#Remoção dos valores inválidos
fF_novo = fF_novo.iloc[1:-3 , :]

In [58]:
fF_novo['DATA'] = pd.to_datetime(fF_novo['DATA'])

fF_novo['Prod. Líquida dia (m³)'].replace('', np.nan, inplace=True)
fF_novo.dropna(subset=['Prod. Líquida dia (m³)'], inplace=True)

fF_novo = fF_novo.reset_index(drop = True)

fF_novo['DATA'] = fF_novo['DATA'].dt.strftime("%Y-%m-%d")

<h1> Agregamento final <h1>

In [124]:
fechamentofiscalGC = pd.concat([fechamento_fiscal_GC, fF_novo])

fechamentofiscalGC['DATA'] = pd.to_datetime(fechamentofiscalGC['DATA'])

#fechamentofiscalGC = fechamentofiscalGC.rename(columns={"Gás Fiscal (20°C e 1atm) (m³):":"GasFiscal","Gás Fiscal de Queima (20°C e 1atm) (m³):":"gasFiscalQueima","Gás Fiscal de Compressão (20°C e 1atm) (m³):":"gasCompressao"})

fechamentofiscalGC['Prod. Líquida dia (m³)'].replace('', np.nan, inplace=True)
fechamentofiscalGC.dropna(subset=['Prod. Líquida dia (m³)'], inplace=True)

fechamentofiscalGC['MovingAvarege'] = fechamentofiscalGC.iloc[:,1].rolling(window=60).mean()

fechamentofiscalGC = fechamentofiscalGC.reset_index(drop = True)

<h1> Fechamento fiscal disponivel! <h1>

In [126]:
#Tratamento dos valores nulos
for i in range (len(fechamentofiscalGC['Prod. Líquida dia (m³)'])):
    if (fechamentofiscalGC['Gás Fiscal de Queima (20°C e 1atm) (m³):'][i] < 0):
        fechamentofiscalGC['Gás Fiscal de Queima (20°C e 1atm) (m³):'][i] = fechamentofiscalGC['Gás Fiscal de Queima (20°C e 1atm) (m³):'][i]*-1        

for i in range (len(fechamentofiscalGC['Prod. Líquida dia (m³)'])):
    if (fechamentofiscalGC['Prod. Líquida dia (m³)'][i] < 0):
        fechamentofiscalGC['Prod. Líquida dia (m³)'][i] = fechamentofiscalGC['Prod. Líquida dia (m³)'][i]*-1
    
fechamentofiscalGC.to_excel('Fechamento_GC.xlsx')
fechamentofiscalGC

,DATA,Prod. Líquida dia (m³),Volume bruto observado (GOV-m³),Temperatura do líquido (°C),Temperatura ambiente (°C),Temperatura de Laboratório (°C),Correção do efeito da temperatura no aço (CTSh),API lido,API ( 60° F),Densidade Corrigida,FCV,BSW %,Água na emulsão (m³),NSV - Volume líquido nas condições padrões 20° C e 1 atm,Gás Fiscal (20°C e 1atm) (m³):,Medição (20°C e 1 atm) (m³),Gás Fiscal de Queima (20°C e 1atm) (m³):,Gás Fiscal de Compressão (20°C e 1atm) (m³):,MovingAvarege
0,2017-10-01,0.000000,5.618000,31.5,33.5,32.5,1.000263,27.4,24.35,0.9159,0.9918,1,0.055734,5.517665,0,0,0,0,NaN
1,2017-10-02,0.000000,5.618000,31.5,33.5,32.5,1.000263,27.4,24.35,0.9159,0.9918,1,0.055734,5.517665,0,0,0,0,NaN
2,2017-10-03,0.000000,5.618000,31.5,33.5,32.5,1.000263,27.4,24.35,0.9159,0.9918,1,0.055734,5.517665,0,0,0,0,NaN
3,2017-10-04,0.000000,5.618000,31.5,33.5,32.5,1.000263,27.4,24.35,0.9159,0.9918,1,0.055734,5.517665,0,0,0,0,NaN
4,2017-10-05,0.000000,5.618000,31.5,33.5,32.5,1.000263,27.4,24.35,0.9159,0.9918,1,0.055734,5.517665,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,2021-07-26,8.125161,23.453799,23.5,23.0,23.0,1.000077,27.1,25.76,0.903,0.9963,0.7,0.163569,23.203451,12622.46,21.862,26.01644,12596.44356,16.622014
1404,2021-07-27,19.602829,21.334786,23.8,23.4,23.4,1.000084,27.1,25.688,0.903,0.9963,2.0,0.425117,20.83073,11254.22,11.39,164.72844,11089.49156,16.607953
1405,2021-07-28,20.167959,18.331703,24.2,23.9,23.9,1.000093,27.1,25.598,0.904,0.9964,2.0,0.365314,17.900395,11625.59,21.976,205.39546,11420.19454,16.624765
1406,2021-07-29,18.042018,36.436639,24.5,24.3,24.3,1.000100,27.1,25.526,0.904,0.9964,1.0,0.363055,35.942412,11748.37,22.566,295.69252,11452.67748,16.635651


In [16]:
fechamento_fiscal_GC = pd.read_excel('Fechamento Fiscal GC.xlsx')

In [19]:
fechamentofiscalGC = pd.concat([fechamento_fiscal_GC, fF_novo])
fechamentofiscalGC.to_excel('Fechamento_GC.xlsx')